## Genearete labels

In [2]:
import pandas as pd
import nutils
import common as cm
import numpy as np
import os
from tqdm import tqdm


def calculate_stats(df, current, future):
    log_returns = np.log(df["mid_price"].shift(-60) / df["mid_price"]) * 1e4
    df[f"mean_{current}-{future}"] = (
        (df["mid_price"].rolling(future - current).mean().shift(-future))
        / df["mid_price"]
    )
    # df[f"var_{current}-{future}"] = (
    #     log_returns.rolling(future - current).var().shift(-future)
    # )
    # df[f"vol_{current}-{future}"] = (
    #     log_returns.rolling(future - current).std().shift(-future)
    # )


def calculate_stats2(df, current, future):
    df[f"max_{current}-{future}"] = (
        (df["mid_price"].rolling(future - current).max().shift(-future))
        / df["mid_price"]
        * 1e4
    )
    df[f"min_{current}-{future}"] = (
        df["mid_price"].rolling(future - current).min().shift(-future)
    ) * 1e4
    df[f"gap_{current}-{future}"] = (
        (
            df["mid_price"].rolling(future - current).max().shift(-future)
            - df["mid_price"].rolling(future - current).min().shift(-future)
        )
        / df["mid_price"]
        * 1e4
    )


for code in tqdm(cm.SELECTED_CODES):
    datas = cm.get_snapshot(code)
    df = datas["tickData"]
    df = pd.DataFrame(df, columns=cm.COLS_SNAPSHOTS)

    # 保存原始列名
    original_columns = set(df.columns)
    df["mid_price"] = (df["AskPrice1"] + df["BidPrice1"]) / 2
    cur_futs = [(0, 60), (60, 120), (60, 300), (60, 600), (60, 6000), (120, 180)]
    for cur,futs in cur_futs:
        calculate_stats(df, cur, futs)
        # calculate_stats2(df, cur, futs)


    # 只保留新生成的列
    new_columns = set(df.columns) - original_columns
    df = df.fillna(0)

    # 指定保存路径
    save_path = f"/mnt/disk1/multiobj_dataset/{code}/"
    os.makedirs(save_path, exist_ok=True)
    for col in new_columns:
        feature_path = os.path.join(save_path, f"{col}.npy")
        np.save(feature_path, df[col].values.reshape(-1, 1))
    print(f"Processed and saved features for {code}")
print(new_columns)

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:03<05:44,  3.48s/it]

Processed and saved features for 000537


  2%|▏         | 2/100 [00:06<05:37,  3.44s/it]

Processed and saved features for 000627


...

Processed and saved features for 688678


 99%|█████████▉| 99/100 [05:10<00:02,  2.76s/it]

Processed and saved features for 688777


100%|██████████| 100/100 [05:14<00:00,  3.15s/it]

Processed and saved features for 689009
{'mid_price', 'mean_120-180', 'mean_60-120', 'mean_0-60', 'mean_60-300', 'mean_60-6000', 'mean_60-600'}


In [4]:
import SharedArray as sa
import numpy as np
code = "000537"
factor = sa.attach(f"factor_{code}")
label = sa.attach(f"label_{code}")
new = np.load(f"/mnt/disk1/multiobj_dataset/{code}/mean_ret_1-60.npy")

In [5]:
new.shape

(3020081, 1)

In [6]:
factor.shape

(1828412, 101)

In [3]:
import SharedArray as sa
import numpy as np
import pandas as pd

In [4]:
a = np.load("/mnt/nas/data/WY/factors/000537.npy")
df = pd.DataFrame(a)

In [6]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,580,581,582,583,584,585,586,587,588,589
count,1.828412e+06,0.0,1.828412e+06,1.828412e+06,0.0,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06,...,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06,1.828412e+06
mean,8.078326e-09,NaN,-4.018758e-07,3.861406e-08,NaN,1.392510e-06,1.668341e-06,7.261053e-08,-2.289972e-08,-3.625232e-08,...,-1.355289e-08,-7.143643e-09,2.053946e-06,2.854119e-09,-9.079771e-09,-1.900234e-06,3.177920e-08,1.462110e-08,6.662950e-08,7.557708e-06
std,9.995817e-01,NaN,9.990209e-01,9.996343e-01,NaN,9.992192e-01,9.993298e-01,9.981529e-01,9.994155e-01,9.995859e-01,...,9.996270e-01,9.996246e-01,9.995074e-01,9.996243e-01,9.996632e-01,9.995623e-01,9.996977e-01,9.997513e-01,9.996541e-01,9.996512e-01
min,-3.633564e+02,NaN,-3.073803e+01,-1.798012e+02,NaN,-2.136823e+01,-2.386577e+01,-4.081592e+01,-2.815655e+01,-1.560041e+01,...,-6.248907e+00,-6.203230e+00,-2.202460e+01,-6.196578e+00,-6.210800e+00,-4.492092e+00,-6.124707e+00,-6.274611e+00,-6.087548e+00,-4.420078e+00
25%,-1.179513e-02,NaN,-4.126149e-01,3.015391e-02,NaN,-3.593167e-01,-5.213097e-01,-1.726747e-01,-5.784067e-01,-7.402822e-01,...,-8.040494e-01,-7.990317e-01,-7.200049e-01,-7.696555e-01,-7.992922e-01,-7.611341e-01,-8.004218e-01,-7.699338e-01,-7.996075e-01,-7.503179e-01
50%,1.277012e-03,NaN,-3.846670e-02,3.248612e-02,NaN,1.403104e-01,8.775415e-02,1.722196e-02,3.622863e-03,-4.571700e-03,...,-1.355238e-02,-1.525443e-02,-3.691749e-02,1.595721e-03,-6.812765e-03,-2.963227e-02,-7.135521e-03,1.899262e-03,-1.059111e-02,-3.368694e-02
75%,1.397133e-02,NaN,3.456947e-01,3.248612e-02,NaN,6.354200e-01,6.928920e-01,1.707401e-01,5.932035e-01,7.430637e-01,...,7.976262e-01,7.927510e-01,6.554657e-01,7.384548e-01,7.902108e-01,7.181430e-01,7.899731e-01,7.416373e-01,7.877862e-01,7.122362e-01
max,5.499131e+02,NaN,5.472511e+01,2.600550e+02,NaN,6.551997e+00,1.421982e+01,4.233598e+01,3.028057e+01,1.575854e+01,...,6.247550e+00,6.024495e+00,4.531549e+01,8.586280e+00,6.536832e+00,5.383172e+01,1.071835e+01,8.722362e+00,9.166877e+00,6.667657e+01
